In [1]:
import pandas as pd
import pandas_datareader.data as web
import datetime as dt
import pymysql

## Define database connector

In [11]:
class db_helper:
    def __init__(self, host='localhost', user='root', password='root', db='db'):
        self.connection = pymysql.connect(host=host,
                         user=user,
                         password=password,
                         db=db)
        self.cursor = self.connection.cursor()
        
    def table_exists(self, table):
        results = self.cursor.execute("SHOW TABLES")
        tables = list()
        for res in self.cursor.fetchall():
            tables.append(*res)
        if table.lower() in tables:
            return True
        return False
    
    def create_table(self, table):
        query = f'''CREATE TABLE `test_ticker`.`{table}` (
                  `ID` INT NOT NULL AUTO_INCREMENT,
                  `High` DOUBLE NULL,
                  `Low` DOUBLE NULL,
                  `Open` DOUBLE NULL,
                  `Close` DOUBLE NULL,
                  `Volume` DOUBLE NULL,
                  `Adj Close` DOUBLE NULL,
                  PRIMARY KEY (`ID`));'''
        self.cursor.execute(query)
        
    def insert_into_table(self, table, cols, data):
        sql = f"INSERT INTO {table} ({cols}) VALUES (" + "%s,"*(len(data)-1) + "%s)"
        try:
            self.cursor.execute(sql, data)
        except Exception as e:
            print(e)
            return
        self.connection.commit()
        
    def df_to_db(self, df, table):
        # Get colum names as string
        cols = ','.join([f"`{col}`" for col in df.columns.to_list()])
        # Insert values into DB, no need for ID value
        for i,row in df.iterrows():
            self.insert_into_table(table, cols, tuple(row))

## Create connection to database

In [12]:
db = db_helper(db='test_ticker')

## Dowload tickers from Yahoo and store in database

In [13]:
# Defines the time periods to use
start = dt.datetime(2019, 1, 1)
end = dt.datetime(2019, 2, 1)

# Reads data into a dataframe
tickets = ['FB', 'VOO']

for ticker in tickers:
    try:
        df = web.DataReader(ticker, 'yahoo', start, end)
    except Exception as e:
        print(e)
        print(f"Ticker '{ticker}' failed to be downloaded")
        continue
    if not db.table_exists(ticker):
        db.create_table(ticker)
    db.df_to_db(df, ticker)